# Web scraping

## Step 1: Install dependencies
Make sure you have the Python `Pandas` and `requests` packages installed to be able to work on the project. In case you do not have the libraries installed, run them in the console:

```console
pip install pandas requests
```
or

```console
conda install pandas requests
```

In [1]:
#TODO

## Step 2: Download HTML
The download of the HTML of the web page will be done with the `requests` library, as we saw in the module theory.

The web page we want to scrape is the following: https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue. It collects and stores information about the growth of the company every three months, since June 2009. It stores the text scraped from the web in some variable.

In [16]:
# TODO

## Step 3: Transform the HTML
The next step to start extracting the information is to transform it into a structured object. Do this using `BeautifulSoup`. Once you have interpreted the HTML correctly, parse it to:

1. Find all the tables.
2. Find the table with the quarterly evolution.
4. Store the data in a DataFrame.

In [3]:
# TODO

## Step 4: Process the DataFrame
Next, clean up the rows to get clean values by removing `$` and commas. Remove also those that are empty or have no information.

In [15]:
# TODO 

## Step 5: Store the data in sqlite
Create an empty instance of the database and include the clean data in it, as we saw in the database module. Once you have an empty database:

1. Create the table.
2. Insert the values.
3. Store (`commit`) the changes.

In [14]:
# TODO

## Step 6: Visualize the data
What types of visualizations can we make? Suggest at least 3 and plot them.

In [17]:
# TODO

In [18]:
# TODO

In [19]:
# TODO